In [1]:
import sys
sys.path.insert(0,'../classes')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.set_printoptions(precision=3,suppress=True)

In [2]:
import logging
import numpy as np

import coloredlogs

from arena import Arena
from coach import Coach
from beck.beck_game import BeckGame as Game
from beck.beck_nnet import NNetWrapper as nn
from beck.beck_players import HumanBeckPlayer, NNPolicyPlayer, NNValuePlayer
from mcts import MCTS
from utils import *
log = logging.getLogger(__name__)

from keras import backend as K

import tournament
participant_iters = tournament.participant_iters

participants_dir = '/scratch/zz737/fiar/tournaments/tournament_4' # dir for trained networks
results_dir = '/scratch/zz737/fiar/tournaments/results/tournament_4'
moves_dir = '/scratch/zz737/fiar/tournaments/tournament_4/moves/raw/'

{'mcts100_cpuct1': [1, 2, 3, 4, 6, 7, 9, 11, 12, 16, 18, 21, 25, 28, 29, 30, 37, 38, 39, 45, 47], 'mcts100_cpuct2': [1, 2, 3, 4, 6, 8, 12, 14, 15, 16, 21, 22, 24, 26, 27, 30, 32, 35, 39], 'mcts100_cpuct3': [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 16, 17, 18, 20, 21, 25, 26, 28, 32, 34, 35, 36], 'mcts25_cpuct1': [1, 2, 3, 4, 9, 11, 13, 14, 16, 19, 22, 25, 28, 29, 31, 37, 39, 45, 59, 61], 'mcts50_cpuct1': [1, 2, 3, 4, 5, 6, 7, 9, 10, 13, 17, 18, 20, 21, 23, 25, 28, 29, 31, 37, 39, 41, 44, 45, 47, 48, 53], 'mcts80_cpuct1': [1, 2, 3, 4, 5, 7, 8, 9, 10, 13, 16, 19, 20, 21, 23, 24, 28, 34, 35, 36, 40], 'mcts80_cpuct2': [1, 2, 5, 6, 8, 10, 12, 13, 15, 17, 18, 19, 22, 25, 28, 30, 31, 34, 37, 38, 39, 40, 43, 45, 46, 51], 'mcts80_cpuct3': [1, 2, 5, 6, 7, 9, 12, 13, 14, 19, 21, 22, 23, 24, 26, 27, 28, 35, 36, 42, 43, 47, 49, 50, 51, 53, 54, 55, 56, 57]}
189 participant iterations!


In [3]:
g = Game(4, 9, 4)
test_board = g.getInitBoard()
test_board[0:3,4]=-1
test_board[0:2,5]=1
print(test_board)

[[ 0  0  0  0 -1  1  0  0  0]
 [ 0  0  0  0 -1  1  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]


In [4]:
test_board = np.random.randint(3,size=((4,9)))-1

In [5]:
test_board

array([[-1, -1,  0,  1, -1,  1,  1,  0,  1],
       [ 0,  0,  1,  0, -1, -1, -1,  0, -1],
       [ 1,  0,  0, -1, -1,  1,  1, -1, -1],
       [ 1,  0,  1,  0,  0,  0,  1, -1,  0]])

In [6]:
test_board = np.zeros_like(test_board)
test_board[0,0]=1
test_board[1,1]=1
test_board[2,2]=0

test_board[0,2]=-1
test_board[1,1]=1
test_board[2,0]=-1

test_board

array([[ 1,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [7]:
from cog_related import cog_value_net as cvn
import importlib
importlib.reload(cvn)

<module 'cog_related.cog_value_net' from '../classes/cog_related/cog_value_net.py'>

In [8]:


def display_to_txt(fn,board):
    square_content = {
        -1: "X",
        +0: "-",
        +1: "O"
    }
    with open(fn,'w') as fo:
        m = board.shape[0]
        n = board.shape[1]
        print("   ", end="",file=fo)
        for y in range(n):
            print(y, end=" ",file=fo)
        print("",file=fo)
        print("-----------------------",file=fo)
        for y in range(m):
            print(y, "|", end="",file=fo)    # print the row #
            for x in range(n):
                piece = board[y][x]    # get the piece to print
                print(square_content[piece], end=" ",file=fo)
            print("|",file=fo)

        print("-----------------------",file=fo)
fn = 'temp_moves.txt'
display_to_txt_with_fn = lambda b:display_to_txt(fn,b)


In [68]:
g = Game(4, 9, 4)
participants_dir_one = os.path.join(participants_dir,'checkpoints_mcts100_cpuct2_id-37549660')
nmcts_p, nnet70 = tournament.get_player(g, participants_dir_one, 'mcts100_cpuct2;70')
nnet = nnet70
args = dotdict({'numMCTSSims':100,
                'cpuct':2
               })
nmcts = MCTS(g, nnet, args)

# w = np.array([0.1,0.2,0.2,1,10])*10
# w = np.array([0.1,0.5,0.5,1,10])*10
w = np.array([0.1,1/3,1/3,5,10])*10
# w = np.array([0.1,0.2,0.5,1,10])*10
C = 0.1#0.1
nnet  = nnet70
cv_net = cvn.NNetWrapper(g,nnet,[w,C])
cvnmcts = MCTS(g, cv_net, args)

ai_nn = lambda x: np.argmax(nmcts.getActionProb(x, temp=0)) #temp=0
ai_cv = lambda x: np.argmax(cvnmcts.getActionProb(x, temp=0)) #temp=0

# arena = Arena(ai_nn, ai_cv, g, display=g.display)
arena = Arena(ai_cv, ai_nn, g, display=g.display)
# arena = Arena(ai_cv, ai_nn, g, display=display_to_txt_with_fn)

# nnet = [nnet70,nnet30]
# nnet_l = [nnet70,cv_net]
nnet_l = [cv_net,nnet70]

human_player = HumanBeckPlayer(g)
human = lambda x: human_player.play(x)
# arena = Arena(ai_cv_noprior, human, g, display=g.display)

# arena.playGame(verbose=True,nnet=nnet_l, is_save_moves=False)

In [27]:
w = np.array([0.1,0.2,0.2,1,10])*10
C = 0.1
cv_noprior = cvn.NNetWrapper(g,None,[w,C])
cvmcts_noprior = MCTS(g, cv_noprior, args)
ai_cv_noprior = lambda x: np.argmax(cvmcts_noprior.getActionProb(x, temp=0)) #temp=0

# arena = Arena(ai_nn, ai_cv, g, display=g.display)
arena = Arena(ai_cv_noprior, ai_nn, g, display=g.display)


In [69]:
oneWon, twoWon, draws, moves_result_multigame = arena.playGames(10,verbose=True, nnet=nnet_l, is_save_moves=True)

Arena.playGames (1):   0%|          | 0/5 [00:00<?, ?it/s]

Value of player to go: 0.0
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: [0.053]
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - O - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: 0.26213130794404793
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - X - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - O - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: [0.583]
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - X - - - - |
1 |- - - - - - - - - |
2 |- - - - O - - O - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: 2.172688588844056
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - X - - - - |
1 |- - - - - - - -

Arena.playGames (1):  20%|██        | 1/5 [01:01<04:04, 61.06s/it]

Value of player to go: 1.4732966803424343
Turn  21 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - X O O O X |
1 |- X - X O O X O - |
2 |- - X O O X - O - |
3 |- O X X - - - X - |
-----------------------
Value of player to go: [-0.058]
Turn  22 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - X O O O X |
1 |- X - X O O X O - |
2 |- - X O O X - O - |
3 |- O X X O - - X - |
-----------------------
Game over: Turn  22 Result  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X - - - X O O O X |
1 |- X - X O O X O - |
2 |- - X O O X - O - |
3 |- O X X O - - X - |
-----------------------
Value of player to go: 0.0
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: [0.143]
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - 

Arena.playGames (1):  40%|████      | 2/5 [01:32<02:36, 52.31s/it]

...

Game over: Turn  15 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O X - X - |
1 |- - X - X O - - - |
2 |- - - O X O O - - |
3 |- - X - O O X O - |
-----------------------
Value of player to go: 0.0
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: [-0.309]
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Value of player to go: 0.4666666666666666
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - X - - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Value of player to go: [-0.269]
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - X O - - - |
2 |- - - - - - - - - |


Arena.playGames (2):  80%|████████  | 4/5 [03:30<00:50, 50.90s/it]

Game over: Turn  11 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - - X O - - - |
2 |- - - X X X O - - |
3 |- - - - O X O O - |
-----------------------
Value of player to go: 0.0
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: [-0.309]
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Value of player to go: 0.4666666666666666
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - X - - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Value of player to go: [-0.278]
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O X - - - - |
2 |- - - - - - - - - |


Arena.playGames (2): 100%|██████████| 5/5 [04:02<00:00, 48.57s/it]

Value of player to go: 15.416801381161942
Turn  11 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O O - - - - |
1 |- - - O X - - - - |
2 |- - X O X X - - - |
3 |- - - X O - - - - |
-----------------------
Value of player to go: [-1.]
Turn  12 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - O O O - - - - |
1 |- - - O X - - - - |
2 |- - X O X X - - - |
3 |- - - X O - - - - |
-----------------------
Value of player to go: 109.101627589488
Turn  13 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - O O O - - - - |
1 |- - X O X - - - - |
2 |- - X O X X - - - |
3 |- - - X O - - - - |
-----------------------
Game over: Turn  13 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- O O O O - - - - |
1 |- - X O X - - - - |
2 |- - X O X X - - - |
3 |- - - X O - - - - |
-----------------------


In [67]:
print(oneWon,twoWon,draws)

0 1 9


In [41]:
arena.player1(board)

23

In [44]:
arena.player2(-board)

20

In [42]:
23//9,23%9

(2, 5)

In [33]:
test_board = "\
0 |- X O O O X - - - | \
1 |- - O - - - - - - | \
2 |- - - X X - - - - | \
3 |- - - - - - - - - |"

board = np.array(test_board.split()).reshape(4,-1)
board = board[:,1:-1]

board[(board=='|-') | (board=='-')] = 0
board[board=='X']=-1
board[board=='O']=1
board = board.astype(np.int)
board

print('normal')
print(np.array(cvnmcts.getActionProb(board)).reshape(4,9))

ind = ai_cv(board)
print('normal', ind//9, ind%9)

ind = ai_cv_noprior(board)
print('no prior', ind//9, ind%9)


print('no prior')
print(np.array(cvmcts_noprior.getActionProb(board)).reshape(4,9))

import copy
test_board = copy.copy(board)
test_board[2,5] = -1
test_board *= -1
print(cv_net.predict(test_board))


normal
[[0.002 0.    0.    0.    0.    0.    0.002 0.002 0.002]
 [0.002 0.002 0.    0.002 0.002 0.002 0.002 0.002 0.002]
 [0.002 0.002 0.844 0.    0.    0.104 0.002 0.002 0.002]
 [0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002]]
normal 2 2
no prior 2 2
no prior
[[0.001 0.    0.    0.    0.    0.    0.001 0.001 0.001]
 [0.001 0.001 0.    0.071 0.001 0.001 0.001 0.001 0.001]
 [0.001 0.001 0.891 0.    0.    0.001 0.001 0.001 0.001]
 [0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]]
(array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.515, 0.   , 0.   , 0.   , 0.485, 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
      dtype=float32), 32.031053163928576)


In [15]:
test_board

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0]])

In [16]:
cvn.filter_all_ortho_diag(-test_board)

{'diag': {'2_con': 0, '2_uncon': 0, '3': 0, '4': 0},
 'ortho': {'2_con': 0, '2_uncon': 0, '3': 0, '4': 0}}

In [ ]:
cvn.filter_all_ortho_diag(-test_board)

In [19]:
test_board = copy.copy(board)
test_board[2,5] = -1
test_board *= -1
# print(cv_net.predict(test_board))
print(cv_net_lowC.predict(test_board))
# print(cvn.filter_all_ortho_diag(test_board))
# print(cvn.filter_all_ortho_diag(-test_board))

test_board = copy.copy(board)
test_board[1,2] = -1
test_board *= -1
# print(cv_net.predict(test_board))
# print(cvn.filter_all_ortho_diag(test_board))
# print(cvn.filter_all_ortho_diag(-test_board))
print(cv_net_lowC.predict(test_board))

(array([0.003, 0.009, 0.004, 0.003, 0.012, 0.005, 0.002, 0.003, 0.002,
       0.002, 0.002, 0.008, 0.029, 0.023, 0.079, 0.006, 0.002, 0.002,
       0.002, 0.002, 0.003, 0.   , 0.   , 0.   , 0.576, 0.076, 0.005,
       0.006, 0.004, 0.007, 0.054, 0.   , 0.062, 0.001, 0.005, 0.002],
      dtype=float32), 4.538317805233257)
(array([0.003, 0.002, 0.023, 0.009, 0.004, 0.002, 0.008, 0.002, 0.004,
       0.004, 0.009, 0.001, 0.522, 0.143, 0.007, 0.006, 0.003, 0.004,
       0.004, 0.006, 0.023, 0.   , 0.   , 0.016, 0.007, 0.005, 0.003,
       0.005, 0.01 , 0.037, 0.058, 0.   , 0.054, 0.01 , 0.005, 0.003],
      dtype=float32), 2.3289618643060077)


In [152]:
import copy
test_board = copy.copy(board)
test_board[1,2] = -1
test_board *= -1
print(test_board)
cv_net.predict(test_board)


[[ 0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0]]


(array([0.003, 0.002, 0.023, 0.009, 0.004, 0.002, 0.008, 0.002, 0.004,
        0.004, 0.009, 0.001, 0.522, 0.143, 0.007, 0.006, 0.003, 0.004,
        0.004, 0.006, 0.023, 0.   , 0.   , 0.016, 0.007, 0.005, 0.003,
        0.005, 0.01 , 0.037, 0.058, 0.   , 0.054, 0.01 , 0.005, 0.003],
       dtype=float32),
 0.9239773924060837)

In [61]:
cv_net.predict(board)

(array([0.   , 0.   , 0.   , 0.734, 0.257, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.007, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ],
       dtype=float32),
 7.749656882450863)

In [59]:
test_board = "\
0 |- X X - - - - - - | \
1 |- - O X - - - - - | \
2 |- X O O O X - - - | \
3 |- - - - O - - - - |"

board = np.array(test_board.split()).reshape(4,-1)
board = board[:,1:-1]

board[(board=='|-') | (board=='-')] = 0
board[board=='X']=-1
board[board=='O']=1
board = board.astype(np.int)
board

cvnmcts_new = MCTS(g, cv_net, args)
print(np.array(cvnmcts.getActionProb(-board)).reshape(4,9))
print(np.array(cvnmcts_new.getActionProb(-board)).reshape(4,9))

ind = ai_cv(-board)
print(ind//9, ind%9)

print(np.array(cvmcts_noprior.getActionProb(board)).reshape(4,9))

import copy
# test_board = copy.copy(board)
# test_board[2,5] = -1
# test_board *= -1
# print(cv_net.predict(test_board))


[[0.003 0.    0.    0.144 0.776 0.003 0.003 0.003 0.003]
 [0.003 0.003 0.    0.    0.003 0.003 0.003 0.003 0.003]
 [0.003 0.    0.    0.    0.    0.    0.003 0.003 0.003]
 [0.003 0.003 0.003 0.003 0.    0.003 0.003 0.003 0.003]]
[[0.01  0.    0.    0.01  0.747 0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.    0.    0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.    0.    0.    0.    0.    0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.    0.01  0.01  0.01  0.01 ]]
0 4
[[0.005 0.    0.    0.874 0.005 0.005 0.005 0.005 0.005]
 [0.005 0.005 0.    0.    0.005 0.005 0.005 0.005 0.005]
 [0.005 0.    0.    0.    0.    0.    0.005 0.005 0.005]
 [0.005 0.005 0.005 0.005 0.    0.005 0.005 0.005 0.005]]


In [126]:
oneWon, twoWon, draws = arena.playGames(10,verbose=True, nnet=nnet_l, is_save_moves=False)

Arena.playGames (1):   0%|          | 0/5 [00:00<?, ?it/s]

Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - O - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - X - - - - - |
2 |- - O - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - O X - - - - - |
2 |- - O - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - X - - - - - - |
1 |- - O X - - - - - |
2 |- - O - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  6 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - X - - - - - - |
1 |- - O X

Arena.playGames (1):  20%|██        | 1/5 [00:31<02:05, 31.48s/it]

Game over: Turn  14 Result  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O X X X X - - - - |
1 |- - O X - - - - - |
2 |- X O O O X - - - |
3 |- - - O O - - - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - O - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - X - - - - - |
2 |- - O - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - O X - - - - - |
2 |- - O - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - X - - - - - - 

Arena.playGames (1):  40%|████      | 2/5 [01:18<01:48, 36.15s/it]

...

Game over: Turn  7 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- O - - - - - - - |
1 |- - O - - - - - - |
2 |- - - O X X - X - |
3 |- - - - O - - - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - X - - - - |
3 |- - - - O - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - O X - - - - |
3 |- - - - O - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |


Arena.playGames (2):  80%|████████  | 4/5 [01:03<00:16, 16.98s/it]

Turn  6 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - O - - - - - - |
2 |- - - O X X - - - |
3 |- - - - O - - - - |
-----------------------
Turn  7 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - O - - - - - - |
2 |- - - O X X - X - |
3 |- - - - O - - - - |
-----------------------
Game over: Turn  7 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- O - - - - - - - |
1 |- - O - - - - - - |
2 |- - - O X X - X - |
3 |- - - - O - - - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |


Arena.playGames (2): 100%|██████████| 5/5 [01:14<00:00, 14.96s/it]

Turn  6 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - O - - - - - - |
2 |- - - O X X - - - |
3 |- - - - O - - - - |
-----------------------
Turn  7 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - O - - - - - - |
2 |- - - O X X - X - |
3 |- - - - O - - - - |
-----------------------
Game over: Turn  7 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- O - - - - - - - |
1 |- - O - - - - - - |
2 |- - - O X X - X - |
3 |- - - - O - - - - |
-----------------------


In [127]:
twoWon

7

In [128]:
oneWon

0

In [129]:
draws

3

# play with value player

In [75]:
import beck.beck_players as bbp
import importlib
importlib.reload(bbp)

participants_dir_one = os.path.join(participants_dir,'checkpoints_mcts100_cpuct2_id-37549660')
nmcts_p, nnet70 = tournament.get_player(g, participants_dir_one, 'mcts100_cpuct2;70')
nmcts_p, nnet30 = tournament.get_player(g, participants_dir_one, 'mcts100_cpuct2;30')

# nnet = [nnet70,nnet30]
nnet = [nnet30,nnet70]

nnvp70 = bbp.NNValuePlayer(g,nnet70)
nnvp30 = bbp.NNValuePlayer(g,nnet30)
# choice, v_l = nnvp.play(test_board,-1)

ai_nonmcts_70 = lambda x:nnvp70.play(x,-1)[0] #return (action, list of values)
# ai_nonmcts_30 = lambda x:nnvp30.play(x,1)[0] #return (action, list of values)
ai_nonmcts_30 = lambda x:nnvp30.play(x,-1)[0] #return (action, list of values)

human_player = HumanBeckPlayer(g)
human = lambda x: human_player.play(x)

g = Game(4, 9, 4)
# arena = Arena(human, ai_nonmcts, g, display=g.display)
# arena = Arena(ai_nonmcts_70, ai_nonmcts_30, g, display=g.display)
# arena = Arena(ai_nonmcts_30, ai_nonmcts_70, g, display=g.display)
arena = Arena(human, ai_nonmcts_30, g, display=g.display)

arena.playGame(verbose=True,nnet=nnet, is_save_moves=False)
# arena.playGames(4,nnet=nnet)

Value of player to go: [0.424]
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 4] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [ 1 4] [ 1 5] [ 1 6] [ 1 7] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 3] [ 2 4] [ 2 5] [ 2 6] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 3] [ 3 4] [ 3 5] [ 3 6] [ 3 7] [ 3 8] 4 1
Invalid move
1 4
Value of player to go: [0.046]
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - O - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: [-0.334]
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - O - - - - |
2 |- - - - - - - - - |
3 |- - - - X - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 4] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [

KeyboardInterrupt: Interrupted by user

In [24]:
import play_ai
import importlib
importlib.reload(play_ai)
play_ai.main()

2021-04-01 14:00:55 cm01.nyu.cluster play_ai[3902741] INFO Loading BeckGame...
2021-04-01 14:00:55 cm01.nyu.cluster play_ai[3902741] INFO Loading NNetWrapper...
2021-04-01 14:00:55 cm01.nyu.cluster play_ai[3902741] INFO Loading checkpoint "/scratch/zz737/fiar/tournaments/tournament_4/checkpoints_mcts100_cpuct2_id-37549660//checkpoint_70.pth.tar"...
2021-04-01 14:00:56 cm01.nyu.cluster play_ai[3902741] INFO Starting the game 🎉


Value of player to go: [0.426]
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 4] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [ 1 4] [ 1 5] [ 1 6] [ 1 7] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 3] [ 2 4] [ 2 5] [ 2 6] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 3] [ 3 4] [ 3 5] [ 3 6] [ 3 7] [ 3 8] 2 4
Value of player to go: [0.074]
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - O - - - - |
3 |- - - - - - - - - |
-----------------------
Value of player to go: [0.757]
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- X - - O - - - - |
3 |- - - - - - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 4] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [ 1 4] [ 1 5] [ 1 6

In [4]:
g = Game(4, 9, 4)
board = g.getInitBoard()
black_pieces = np.array([[0,1],[0,4],[0,5],[1,3],[1,4],[1,5],[1,6],[2,4],[2,6],[3,2]])
white_pieces = np.array([[0, 3],[1,2],[2,2],[2,3],[2,5],[2,7],[3,3],[3,4],[3,7]])
board[black_pieces[:,0],black_pieces[:,1]] = 1
board[white_pieces[:,0],white_pieces[:,1]] = -1

In [ ]:
# test_board = g.getInitBoard()
# test_board = np.tile(test_board,(2,1,1))
test_board = randb
nnet.predict_batch(test_board)

In [ ]:
K.clear_session()

In [ ]:
keys = list(v_l_flpud_dict.keys())
keys = keys[5:9]
plt.figure()
for k in keys:
    plt.plot(v_l_flpud_dict[k],label=k)
plt.legend()

In [ ]:
keys = list(v_l_dict.keys())
keys = keys[5:9]
plt.figure()
for k in keys:
    plt.plot(v_l_dict[k],label=k)
plt.legend()

In [ ]:
participant_iters

In [ ]:
def get_iters_test_board_value(test_board,iters,game):
    agent_keys = iters.keys()
    v_l_dict = {}
    for k in agent_keys:
        p_iters = [f'{k};{x}' for x in iters[k]]
        v_l = get_participant_iters_test_board_value(test_board,p_iters,game)
        v_l_dict[k] = v_l
    return v_l_dict
        

def get_participant_iters_test_board_value(test_board,participant_iters,game):
    v_l = []
    for p in participant_iters:
        nmcts, nnet = tournament.get_player(game, participants_dir, p)
        p,v = nnet.predict(test_board)
        v_l.append(v)
        K.clear_session()
    return np.array(v_l)
        

In [ ]:
v_l = get_participant_iters_test_board_value(test_board,participant_iters[:40])

In [ ]:
participant_iters[:40]

In [ ]:
plt.plot(v_l)

In [ ]:
numMCTSSims = 80
cpuct = 1
model_class_name = f'checkpoints_mcts{numMCTSSims}_cpuct{cpuct}'
model_instance_name = 'checkpoint_40'
temp = 0
model_dir = '/scratch/zz737/fiar/tournaments/tournament_1/'

args = dotdict({
    'display':True,
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'temp':temp,
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': numMCTSSims,#100,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': cpuct,

    'checkpoint': './temp/',
    'load_model': True,
    # 'load_folder_file': (f'../models/{model_class_name}',f'{model_instance_name}.pth.tar'),
    'load_folder_file': (f'{model_dir}/{model_class_name}',f'{model_instance_name}.pth.tar'),

    'numItersForTrainExamplesHistory': 20,

    # for saving moves
    'is_save_moves': False,
    'save_moves_folder': f'../moves/{model_class_name}',
    'save_moves_file': f'{model_instance_name}_temp{temp}.csv', 
    'overwrite': True, # whether overwrite the existing csv file or add to it

    #
    'numGames':2,

})


In [ ]:
g = Game(4, 9, 4)

log.info('Loading %s...', nn.__name__)
nnet = nn(g)
nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])

In [17]:
test_board = g.getInitBoard()
test_board[0:3,4]=-1
test_board[0:2,5]=1

In [18]:
test_board

array([[ 0,  0,  0,  0, -1,  1,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  1,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [ ]:
nnet.predict(test_board) # checkpt 1

In [ ]:
nnet.predict(test_board) #checkpt 11

In [ ]:
nnet.predict(test_board) #checkpt 22

In [ ]:
test_board = g.getInitBoard()
test_board[2,4] = 1
nnet.predict(test_board)

In [ ]:
test_board = g.getInitBoard()
test_board[2,4] = -1
nnet.predict(test_board)

In [1]:
import pandas as pd

In [4]:
d=pd.DataFrame(index=['a','b'],columns=['aa'])

In [11]:
pd.DataFrame(0,index=['a','b'],columns=['aa'])

,aa
a,0
b,0


In [12]:
[1,2,3][:1]

[1]